# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,699.599279,-0.888138,1234,698.711141,700.487417,1397.422281,1400.974834
6,433.273149,0.254007,1234,433.527156,433.019142,867.054312,866.038283
7,469.287782,-0.253898,1234,469.033884,469.541680,938.067768,939.083359
8,672.287443,-0.770178,1234,671.517265,673.057620,1343.034530,1346.115241
9,0.853886,-0.120739,1234,0.733147,0.974624,1.466294,1.949248
10,904.831188,0.242500,1234,905.073688,904.588688,1810.147375,1809.177376
11,283.239278,0.190653,1234,283.429931,283.048625,566.859861,566.097250
12,747.666734,-0.954645,1234,746.712089,748.621379,1493.424177,1497.242758
13,999.597573,0.412377,1234,1000.009950,999.185195,2000.019900,1998.370390
14,810.976678,0.326236,1234,811.302914,810.650442,1622.605828,1621.300885


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-698.711141,700.487417
6,-433.527156,433.019142
7,-469.033884,469.541680
8,-671.517265,673.057620
9,-0.733147,0.974624
10,-905.073688,904.588688
11,-283.429931,283.048625
12,-746.712089,748.621379
13,-1000.009950,999.185195
14,-811.302914,810.650442


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-698.711141,700.487417
6,-433.527156,433.019142
7,-469.033884,469.541680
8,-671.517265,673.057620
9,-0.733147,0.974624
10,-905.073688,904.588688
11,-283.429931,283.048625
12,-746.712089,748.621379
13,-1000.009950,999.185195
14,-811.302914,810.650442


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-6.98711141e+02,  7.00487417e+02],
       [-4.33527156e+02,  4.33019142e+02],
       [-4.69033884e+02,  4.69541680e+02],
       [-6.71517265e+02,  6.73057620e+02],
       [-7.33147085e-01,  9.74624159e-01],
       [-9.05073688e+02,  9.04588688e+02],
       [-2.83429931e+02,  2.83048625e+02],
       [-7.46712089e+02,  7.48621379e+02],
       [-1.00000995e+03,  9.99185195e+02],
       [-8.11302914e+02,  8.10650442e+02]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1395.422281,-1400.974834
6,865.054312,-866.038283
7,936.067768,-939.083359
8,1341.034530,-1346.115241
9,-0.533706,-1.949248
10,1808.147375,-1809.177376
11,564.859861,-566.097250
12,1491.424177,-1497.242758
13,1998.019900,-1998.370390
14,1620.605828,-1621.300885


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,True,True,True
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.699599,0.055931
6,0.433273,0.627004
7,0.469288,0.373051
8,0.672287,0.114911
9,0.000854,0.439631
10,0.904831,0.621250
11,0.283239,0.595326
12,0.747667,0.022677
13,0.999598,0.706189
14,0.810977,0.663118


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,699.599279,-0.888138
6,433.273149,0.254007
7,469.287782,-0.253898
8,672.287443,-0.770178
9,0.853886,-0.120739
10,904.831188,0.242500
11,283.239278,0.190653
12,747.666734,-0.954645
13,999.597573,0.412377
14,810.976678,0.326236


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
